In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import math
import time
from model import NeRF
from datasets import StanfordDragonDataset

In [3]:
# Render testing data
cuda = False
device = torch.device("cuda") if (cuda) else torch.device("cpu")
min_bounds = torch.Tensor([-10,-10,-10]).to(device)
max_bounds = torch.Tensor([10, 10, 10]).to(device)
pth_model = NeRF(device, min_bounds, max_bounds)
pth_model.load_state_dict(torch.load("model-0.pth"))
if (cuda): pth_model.cuda()
pth_model.eval()
testing_poses = torch.from_numpy(np.load("./testing_poses.npy"))
dummy_image, dummy_pose, focal = dataset[0]

def render_image(idx):
    with torch.no_grad():
        pose = testing_poses[idx]
        positions, directions, gt_colors = pth_model.get_rays(dummy_image, pose, focal)
        current_idx = 0
        color = torch.zeros(dummy_image.shape).reshape((-1, 3))
        depth = torch.zeros(dummy_image.shape).reshape((-1, 3))
        while(current_idx < positions.shape[0]):
            indices = torch.arange(current_idx, min(positions.shape[0], current_idx + rays_per_batch))
            pos = positions[indices].to(device)
            dirs = directions[indices].to(device)
            gt = gt_colors[indices].to(device)
            colors, depths, _ = pth_model.render_rays(pos, dirs, num_samples)
            color[indices] = colors.float().cpu()
            depth[indices] = depths[..., None].float().cpu()
            current_idx += rays_per_batch
        color = color.reshape(dummy_image.shape) * 255
        depth = depth.reshape(dummy_image.shape) * 255
        pil_color = Image.fromarray(color.numpy().astype(np.uint8))
        pil_color.save("test-color-%d.jpeg" % idx)
        pil_depth = Image.fromarray(depth.numpy().astype(np.uint8))
        pil_depth.save("test-depth-%d.jpeg" % idx)
        
for i in range(testing_poses.shape[0]): render_image(i)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.